In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
from tqdm import tqdm
from sklearn.metrics import f1_score
import random
from torch import nn
import seaborn as sns
import torch.nn.functional as F
from torchvision.models import mobilenet_v3_large, vit_b_32, mobilenet_v2
from sklearn_extra.cluster import KMedoids
from calflops import calculate_flops

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 123

class TokenPooling(nn.Module):
    def __init__(self, keep_tokens: int, use_weighted: bool = True):
        super().__init__()
        self.keep_tokens = keep_tokens
        self.use_weighted = use_weighted

    def forward(self, x: torch.Tensor, significance: torch.Tensor = None) -> torch.Tensor:
        B, N_plus_1, D = x.shape
        cls_token, tokens = x[:, :1, :], x[:, 1:, :]  # (B, 1, D), (B, N, D)

        if self.keep_tokens >= tokens.shape[1]:
            return x  # tidak perlu pooling

        if not self.use_weighted:
            significance = torch.ones(tokens.shape[:2], device=x.device)

        # Ambil top-k token berdasarkan skor
        topk_scores, topk_indices = torch.topk(significance, self.keep_tokens, dim=1)  # (B, K)

        # Ambil token berdasarkan indeks top-k
        B_idx = torch.arange(B, device=x.device).unsqueeze(1).expand(-1, self.keep_tokens)  # (B, K)
        pooled_tokens = tokens[B_idx, topk_indices]  # (B, K, D)

        return torch.cat([cls_token, pooled_tokens], dim=1)  # (B, K+1, D)

class MultiheadSelfAttentionBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 num_heads:int=12,
                 attn_dropout:float=0.):
        super().__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim,
                                                    num_heads=num_heads,
                                                    dropout=attn_dropout,
                                                    batch_first=True)
        self.attn_weights = None
    def forward(self, x):
        attn_output, attn_weights = self.multihead_attn(query=x,
                                             key=x,
                                             value=x,
                                             need_weights=True,
                                             average_attn_weights=False)
        self.attn_weights = attn_weights
        return attn_output, attn_weights
    
class MLPBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 mlp_size:int=3072,
                 dropout:float=0.):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_features=embedding_dim,
                      out_features=mlp_size),
            nn.GELU(),
            nn.Dropout(p=dropout),
            nn.Linear(in_features=mlp_size,
                      out_features=embedding_dim),
            nn.Dropout(p=dropout)
        )
    def forward(self, x):
        x = self.mlp(x)
        return x

class TransformerEncoderBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 num_heads:int=12,
                 mlp_size:int=3072,
                 mlp_dropout:float=0.,
                 attn_dropout:float=0.):
        super().__init__()
        self.layer_norm1 = nn.LayerNorm(normalized_shape=embedding_dim, eps=1e-6)
        
        self.msa_block = MultiheadSelfAttentionBlock(embedding_dim=embedding_dim,
                                                     num_heads=num_heads,
                                                     attn_dropout=attn_dropout)
        
        self.layer_norm2 = nn.LayerNorm(normalized_shape=embedding_dim, eps=1e-6)
        
        self.mlp_block =  MLPBlock(embedding_dim=embedding_dim,
                                   mlp_size=mlp_size,
                                   dropout=mlp_dropout)
    def forward(self, x):
        x_attn, attn_weights = self.msa_block(self.layer_norm1(x))
        x = x_attn + x
        
        x = self.mlp_block(self.layer_norm2(x)) + x 
        
        return x, attn_weights

class ViTMobilenet(nn.Module):
    def __init__(self,
                 img_size:int=224, # Training resolution from Table 3 in ViT paper
                 in_channels:int=3, # Number of channels in input image
                 patch_size:int=16, # Patch size
                 num_transformer_layers:int=12, # Layers from Table 1 for ViT-Base
                 embedding_dim:int=768, # Hidden size D from Table 1 for ViT-Base
                 mlp_size:int=3072, # MLP size from Table 1 for ViT-Base
                 num_heads:int=12, # Heads from Table 1 for ViT-Base
                 attn_dropout:float=0., # Dropout for attention projection
                 mlp_dropout:float=0., # Dropout for dense/MLP layers 
                 embedding_dropout:float=0., # Dropout for patch and position embeddings
                 num_classes:int=1000): # Default for ImageNet but can customize this
        super().__init__()
         
        assert img_size % 32 == 0, f"Image size must be divisible by 32, image size: {img_size}"
        
        self.mobilenet = mobilenet_v3_large(pretrained=True).features
        
        self.projection = nn.Conv2d(in_channels=960, 
                                    out_channels=embedding_dim,
                                    kernel_size=1)
                 
        self.class_embedding = nn.Parameter(data=torch.randn(1, 1, embedding_dim),
                                            requires_grad=True)

        self.num_patches = (img_size // 32) ** 2  # MobileNet reduces spatial size by 32x
        
        self.position_embedding = nn.Parameter(data=torch.randn(1, self.num_patches+1, embedding_dim),
                                               requires_grad=True)
                
        self.embedding_dropout = nn.Dropout(p=embedding_dropout)
        
        self.transformer_encoder = nn.Sequential(*[TransformerEncoderBlock(embedding_dim=embedding_dim,
                                                                            num_heads=num_heads,
                                                                            mlp_size=mlp_size,
                                                                            mlp_dropout=mlp_dropout) for _ in range(num_transformer_layers)])
        
        self.keep_tokens = [40, 35, 30, 26, 23, 20, 12, 12, 6, 6, 3, 0]
        # Tambahkan TokenPooling per layer (jumlah token disesuaikan)
        self.token_pools = nn.ModuleList([
            TokenPooling(keep_tokens=k, use_weighted=True) for k in self.keep_tokens
        ])
       
        self.norm = nn.LayerNorm(normalized_shape=embedding_dim, eps=1e-6)
        self.head = nn.Linear(in_features=embedding_dim, out_features=num_classes)
    
    def forward(self, pixel_values, labels=None):
        
        batch_size = pixel_values.shape[0]

        # Extract features using MobileNet
        features = self.mobilenet(pixel_values)  # Output shape: (batch_size, 1280, H', W')
        features = self.projection(features)  # Project to embedding_dim: (batch_size, embedding_dim, H', W')

        # Flatten the feature maps into a sequence of tokens
        features = features.flatten(2).transpose(1, 2)  # Shape: (batch_size, num_patches, embedding_dim)
        
        class_token = self.class_embedding.expand(batch_size, -1, -1)

        x = torch.cat((class_token, features), dim=1)  # Shape: (batch_size, num_patches + 1, embedding_dim)

        x = x + self.position_embedding

        x = self.embedding_dropout(x)
        
        significance_scores = []

        for i, block in enumerate(self.transformer_encoder):
            x, attn_weights = block(x)
            
            # Hitung significance score: total attention yang diterima setiap token
            score = attn_weights.sum(dim=1).sum(dim=1)[:, 1:]  # shape: (B, N-1)
            
            significance_scores.append(score)
            
            if self.token_pools[i].keep_tokens > 0:
                x = self.token_pools[i](x, significance=score)
            else:
                x = x[:, :1, :]  # hanya CLS token

        x = self.norm(x)
        
        cls_token_final = x[:, 0]

        logits = self.head(cls_token_final)

        return logits

model = ViTMobilenet(num_classes=7, 
                in_channels=3, 
                num_heads=12, 
                embedding_dim=768, 
                num_transformer_layers=12,
                mlp_size=3072)
model.to(DEVICE)

# model = mobilenet_v3_large(pretrained=True)
# model.classifier[3] = nn.Linear(model.classifier[3].in_features, 7)
# model.to(DEVICE)

# model = vit_b_32(pretrained=True)
# model.heads.head = nn.Linear(model.heads.head.in_features, 7)
# model.to(DEVICE)

input_shape = (1, 3, 224, 224)

flops, macs, params = calculate_flops(model=model, 
                                      input_shape=input_shape,
                                      output_as_string=True,
                                      output_precision=4)
print("FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))

C:\Users\MoKha\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\MoKha\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  88.81 M 
fwd MACs:                                                               2.1896 GMACs
fwd FLOPs:                                                              4.3939 GFLOPS
fwd+bwd MACs:                                                           6.5687 GMACs
fwd+bwd FLOPs:                                                          13.1818 GFLOPS

-------------------------------- Detailed Calculated FLOPs Results --------------------------------
Each modul